# Exploratory Data Analysis (EDA) Report
This report analyzes the dataset `a34_1_refused_cleaned.csv`, which includes refusal statistics based on inadmissibility grounds by country, year, and residency status.

In [29]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

df = pd.read_csv('../data/processed/a34_1_refused_cleaned.csv')
df.head()

,inadmissibility_grounds,country,year,cor_status,resident,count
0,A34(1),Afghanistan,2019,COR Not Canada,Permanent Resident,1
1,A34(1),Argentina,2019,COR Not Canada,Permanent Resident,0
2,A34(1),Egypt,2019,COR Not Canada,Permanent Resident,1
3,A34(1),Eritrea,2019,COR Not Canada,Permanent Resident,0
4,A34(1),Haiti,2019,COR Not Canada,Permanent Resident,0


# Data Cleaning

In [30]:
# df["country"].unique()

In [31]:
# country_name_map = {
#     'People\'s Republic of China': 'China',
#     'Democratic Rep. of Congo': 'Congo, Democratic Republic of the',
#     'Federal Republic of Germany': 'Germany, Federal Republic Of',
#     'Kosovo, Republic of': 'Kosovo',
#     'Palestinian Authority (Gaza/West Bank)': 'Palestinian Authority',
# }

# df['country'] = df['country'].replace(country_name_map)


# df.to_csv('../data/processed/a34_1_refused_cleaned.csv', index=False)


## Field Overview
We begin by examining each field for data types, missing values, and unique value distributions.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3486 entries, 0 to 3485
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   inadmissibility_grounds  3486 non-null   object
 1   country                  3486 non-null   object
 2   year                     3486 non-null   int64 
 3   cor_status               3486 non-null   object
 4   resident                 3486 non-null   object
 5   count                    3486 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 163.5+ KB


## Refusal Reasons Distribution

In [33]:
refusal_counts = df.groupby("inadmissibility_grounds")["count"].sum().reset_index()
fig = px.bar(refusal_counts, x="inadmissibility_grounds", y="count", title="Total Refusals by Inadmissibility Grounds")
fig.show()

This bar chart illustrates the total number of refusals under different inadmissibility grounds as defined in **Section 34(1) of Canada’s Immigration and Refugee Protection Act (IRPA)**. Each bar represents a specific clause (e.g., A34(1)(a), A34(1)(b), etc.), and the height of the bar indicates the number of individuals refused entry based on that ground.

---

### Key Observations

* **A34(1)(f)** — *Membership in an inadmissible organization* — is by far the most frequently cited ground, with around 600 refusal cases. This clause applies even if the individual did not directly engage in espionage or terrorism but was merely associated with such an organization.

* Other commonly cited clauses include:

  * **A34(1)(a)**: Espionage or subversion against Canada’s interests.
  * **A34(1)(d)**: Being a danger to the security of Canada.
  * **A34(1)(c)**: Engagement in terrorism.
  * **A34(1)(b)**: Attempts to overthrow a government by force.

* **A34(1)(b.1)** and **A34(1)(e)** are the least cited, suggesting relatively few cases involve attempts to subvert democratic institutions (b.1) or acts of violence threatening individuals (e).


## Total Refusals by Country

In [34]:
country_counts = df.groupby("country")["count"].sum().reset_index().sort_values(by="count", ascending=False)
fig = px.bar(country_counts.head(10), x="country", y="count", title="Top 10 Countries by Total Refusals")
fig.show()

This chart shows the top 10 countries by total refusal cases. **Ukraine** leads by a wide margin, followed by **Syria** and **Iran**. The high numbers may reflect conflict zones or perceived security risks. Countries like **China**, **Russia**, and **India** also appear, but with fewer cases. 


## Refusal Trends Over Time

In [35]:
yearly_trend = df.groupby("year")["count"].sum().reset_index()
fig = px.line(yearly_trend, x="year", y="count", markers=True, title="Total Refusals Per Year")
fig.show()

Refusals dropped sharply in 2020, likely due to pandemic-related disruptions, then gradually increased. A major spike occurred in **2024**, reaching the highest level in the dataset, suggesting a possible policy change or increased enforcement.


## Heatmap: Refusals by Country and Year

In [36]:
top_countries = country_counts.head(10)["country"].tolist()
df_top = df[df["country"].isin(top_countries)]
heatmap_data = df_top.pivot_table(index="country", columns="year", values="count", aggfunc="sum", fill_value=0)
fig = px.imshow(heatmap_data, text_auto=True, aspect='auto', color_continuous_scale='Reds',
                labels=dict(x="Year", y="Country", color="Refusals"),
                title="Heatmap of Refusals for Top 10 Countries by Year")
fig.show()

Ukraine shows a sharp surge in 2024 with **131 refusals**, far exceeding all other countries and years. Syria also had a peak in 2022. Most other countries show relatively stable or low refusal counts over time.


## COR Status and Residency Analysis

In [37]:
cor_counts = df.groupby("cor_status")["count"].sum().reset_index()
fig1 = px.pie(cor_counts, names="cor_status", values="count", title="Refusals by COR Status")
fig1.show()

res_counts = df.groupby("resident")["count"].sum().reset_index()
fig2 = px.pie(res_counts, names="resident", values="count", title="Refusals by Residency Type")
fig2.show()

The majority of refusals (85.3%) involve applicants whose Country of Residence (COR) is not Canada, suggesting most cases affect individuals applying from abroad rather than from within Canada.

Refusals are slightly more common among permanent residents (56.7%) than temporary residents (43.3%), indicating both groups face significant scrutiny under inadmissibility grounds.

## Trend of Refusals Over Time by Top 5 Countries

In [38]:
# Summarize total refusals per country per year
df_country_year = df.groupby(["country", "year"])["count"].sum().reset_index()

# Select the top 5 countries with the highest total number of refusals
top_countries_overall = df_country_year.groupby("country")["count"].sum().nlargest(5).index
df_top_country_year = df_country_year[df_country_year["country"].isin(top_countries_overall)]

# Plot line chart of refusal trends over time
fig_country_trend = px.line(
    df_top_country_year,
    x="year",
    y="count",
    color="country",
    markers=True,
    title="Trend of Refusals Over Time by Top 5 Countries",
    labels={"count": "Number of Refusals", "year": "Year"}
)

fig_country_trend.show()


This line chart illustrates the trend of refusals over time for the top 5 countries: **Ukraine**, **Syria**, **Iran**, **Eritrea**, and **Bangladesh**.
### Key Observations            

* **Ukraine**: Shows a dramatic spike in 2024, with refusals exceeding 130 cases — this could be linked to geopolitical instability or shifts in immigration policy.
* **Syria**: Peaks sharply in 2022 and then drops off, indicating strong temporal fluctuations, possibly tied to policy or humanitarian events.
* **Iran, Eritrea, and Bangladesh**: These countries show relatively stable trends with gradual increases from 2021 to 2024, and no sudden changes.



## Heatmap Commentary: Refusals by Country and Inadmissibility Type


In [39]:
# Create a pivot table for the heatmap: countries as rows, inadmissibility types as columns
df_heatmap = df.groupby(["country", "inadmissibility_grounds"])["count"].sum().reset_index()
pivot_heatmap = df_heatmap.pivot(index="country", columns="inadmissibility_grounds", values="count").fillna(0)

# Plot using go.Heatmap for compatibility and full control
fig_heatmap = go.Figure(data=go.Heatmap(
    z=pivot_heatmap.values,
    x=list(pivot_heatmap.columns),
    y=list(pivot_heatmap.index),
    colorscale='Reds',
    colorbar=dict(title='Number of Refusals')
))

fig_heatmap.update_layout(
    title='Heatmap of Refusals by Country and Inadmissibility Type',
    xaxis_title='Inadmissibility Type',
    yaxis_title='Country'
)

fig_heatmap.show()


This heatmap visualizes the number of refusals by country and inadmissibility type. Each cell represents a specific country and the corresponding refusal count for a particular inadmissibility ground.

### Key Insights:

* **A34(1)(f)** stands out with a strong concentration of refusals across many countries — especially one near the top (likely Uzbekistan or Uganda) — suggesting it's a commonly applied ground.
* Most countries have **very low or no activity** in categories like A34(1) to A34(1)(b.1), possibly reflecting the rarity or narrow scope of those legal provisions.
* A few countries, such as **Ecuador, Iraq, Cuba**, and **Afghanistan**, show broader involvement across multiple inadmissibility types.



## Refusal Trends Over Time by Inadmissibility Types

In [40]:
# Group data by inadmissibility type and year, summing refusals
df_reason_trend = df.groupby(["inadmissibility_grounds", "year"])["count"].sum().reset_index()

# Select inadmissibility types
top_reasons = df_reason_trend.groupby("inadmissibility_grounds")["count"].sum().index
df_top_reason_trend = df_reason_trend[df_reason_trend["inadmissibility_grounds"].isin(top_reasons)]

# Plot line chart
fig_reason_trend = px.line(
    df_top_reason_trend,
    x="year",
    y="count",
    color="inadmissibility_grounds",
    markers=True,
    title="Refusal Trends Over Time by Inadmissibility Types",
    labels={"count": "Number of Refusals", "year": "Year", "inadmissibility_grounds": "Inadmissibility Type"}
)

fig_reason_trend.show()



### Key Observations:

* **A34(1)(f)** (often linked to organized crime or national security threats) dominates in absolute volume and shows a **dramatic surge in 2024**, more than doubling from the previous year. This could reflect **heightened enforcement**, **policy shifts**, or **global conflict spillovers**.
* **A34(1)(a)** and **A34(1)(d)** also rise modestly in 2023–2024, suggesting potentially increased scrutiny under these categories.
* Most other types remain **stable or very low**, including **A34(1)(e)** and **A34(1)(b.1)**, indicating these provisions are either rarely applied or narrowly defined.


## Stacked Bar Chart: Refusals by COR Status and Residency Type

In [41]:
# Group data by COR status and residency type
df_stackbar = df.groupby(["cor_status", "resident"])["count"].sum().reset_index()

# Plot stacked bar chart
fig_stackbar = px.bar(
    df_stackbar,
    x="resident",
    y="count",
    color="cor_status",
    title="Stacked Bar Chart: Refusals by COR Status and Residency Type",
    labels={
        "cor_status": "COR Status",
        "count": "Number of Refusals",
        "resident": "Residency Type"
    },
    barmode="stack"
)

fig_stackbar.show()


### Key Insights:

* **Temporary Residents** show an overwhelming concentration of refusals from **COR Not Canada**, with barely any from **COR Canada**. This suggests that **foreign temporary applicants are much more likely to be refused**.
* **Permanent Residents** are more evenly split, but still show a significantly higher number of refusals from outside Canada.
* This reinforces the idea that **being outside Canada significantly increases refusal likelihood**, regardless of residency type.


## Summary
This EDA highlights patterns in refusal data across time, countries, and different grounds of inadmissibility. Most refusals are concentrated in a few countries and specific inadmissibility reasons. The dataset can be further explored for policy impact or predictive modeling.

# EDA Cont.

In [42]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

df = pd.read_csv('../data/processed/a34_1_refused_cleaned.csv')
df.head()

,inadmissibility_grounds,country,year,cor_status,resident,count
0,A34(1),Afghanistan,2019,COR Not Canada,Permanent Resident,1
1,A34(1),Argentina,2019,COR Not Canada,Permanent Resident,0
2,A34(1),Egypt,2019,COR Not Canada,Permanent Resident,1
3,A34(1),Eritrea,2019,COR Not Canada,Permanent Resident,0
4,A34(1),Haiti,2019,COR Not Canada,Permanent Resident,0


In [43]:
# Separate A34(1)(f)
df_f = df[df["inadmissibility_grounds"] == "A34(1)(f)"]
df_f_grouped = df_f.groupby(["country", "inadmissibility_grounds"])["count"].sum().nlargest(15).reset_index()

# All other grounds
df_other = df[df["inadmissibility_grounds"] != "A34(1)(f)"]
df_other_grouped = df_other.groupby(["country"])["count"].sum().nlargest(15).reset_index()

In [44]:
country_counts = df.groupby("country")["count"].sum().reset_index().sort_values(by="count", ascending=False)
fig = px.bar(country_counts.head(15), x="country", y="count", title="Top 10 Countries by Total Refusals")
fig.show()

In [45]:
fig_f = px.bar(
    df_f_grouped.sort_values("count", ascending=False),
    x="country",
    y="count",
    title="Counts of A34(1)(f) Cases by Country",
    labels={"count": "Number of Cases"},
    color="country"
)
fig_f.update_layout(xaxis_tickangle=-45)
fig_f.show()

In [46]:
fig_f = px.bar(
    df_other_grouped.sort_values("count", ascending=False),
    x="country",
    y="count",
    title="Counts of other Cases by Country",
    labels={"count": "Number of Cases"},
    color="country"
)
fig_f.update_layout(xaxis_tickangle=-45)
fig_f.show()

In [47]:
import plotly.express as px
import pandas as pd

# Assuming df is already loaded
top_countries_total = df.groupby("country")["count"].sum().nlargest(11).index
df_top10 = df[df["country"].isin(top_countries_total)]

fig_top10 = px.treemap(
    df_top10,
    path=["country", "inadmissibility_grounds"],
    values="count",
    title="Treemap: Top 10 Countries by Total Count (All Grounds)",
    hover_data=["count"],
)

fig_top10.update_traces(
    textinfo="label+value+percent entry"
)

fig_top10.update_layout(
    width=1400,
    height=800,
    margin=dict(t=50, l=5, r=5, b=5),
    font=dict(
        size=20
    ),
    title_font_size=24
)

fig_top10.show()

In [48]:
# Filter out A34(1)(f)
df_no_f = df[df["inadmissibility_grounds"] != "A34(1)(f)"]
top_countries_no_f = df_no_f.groupby("country")["count"].sum().nlargest(11).index
df_top10_no_f = df_no_f[df_no_f["country"].isin(top_countries_no_f)]

fig_top10_no_f = px.treemap(
    df_top10_no_f,
    path=["country", "inadmissibility_grounds"],
    values="count",
    title="Treemap: Top 10 Countries by Total Count (Excluding A34(1)(f))",
    hover_data=["count"],
)

fig_top10_no_f.update_traces(
    textinfo="label+value+percent entry"
)
fig_top10_no_f.update_layout(
    width=1400,
    height=800,
    margin=dict(t=50, l=5, r=5, b=5),
    font=dict(
        size=20
    ),
    title_font_size=24
)
fig_top10_no_f.show()

In [49]:
import plotly.graph_objects as go

# Group and calculate
grouped = df.groupby(["inadmissibility_grounds", "resident"])["count"].sum().reset_index()

# Calculate total and percentage
total_per_ground = grouped.groupby("inadmissibility_grounds")["count"].transform("sum")
grouped["percentage"] = (grouped["count"] / total_per_ground) * 100
grouped["text"] = grouped["count"].astype(str) + " (" + grouped["percentage"].round(1).astype(str) + "%)"

# Get unique categories
grounds = grouped["inadmissibility_grounds"].unique()
residents = grouped["resident"].unique()

# Create figure
fig = go.Figure()

for resident in residents:
    df_res = grouped[grouped["resident"] == resident]
    fig.add_trace(go.Bar(
        x=df_res["inadmissibility_grounds"],
        y=df_res["count"],
        name=resident,
        text=df_res["text"],
        textposition="outside",
        textfont=dict(size=20)  # Font size for bar labels
    ))

fig.update_layout(
    barmode='group',
    title="Counts and Percentages by Inadmissibility Grounds and Resident Type",
    title_font=dict(size=24),
    xaxis_title="Inadmissibility Ground",
    yaxis_title="Total Count",
    font=dict(size=16),  # General font size (axes, legend, etc.)
    xaxis=dict(tickfont=dict(size=14), title_font=dict(size=18)),
    yaxis=dict(tickfont=dict(size=14), title_font=dict(size=18)),
    legend=dict(font=dict(size=20)),
    width=1200,
    height=700
)

fig.show()

In [50]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# List of grounds to process
grounds = ["A34(1)(f)", "A34(1)(a)", "A34(1)(b)", "A34(1)(c)", "A34(1)(d)"]

for ground in grounds:
    print(f"Generating visualizations for {ground}...")

    # Filter the ground
    ground_df = df[df["inadmissibility_grounds"] == ground]

    # Top 5 countries by count
    top_countries = ground_df.groupby("country")["count"].sum().nlargest(6).index
    df_top = ground_df[ground_df["country"].isin(top_countries)]

    grouped = df_top.groupby(["country", "resident"])["count"].sum().reset_index()

    # Calculate percentage
    total_per_country = grouped.groupby("country")["count"].transform("sum")
    grouped["percentage"] = (grouped["count"] / total_per_country) * 100
    grouped["text"] = grouped["count"].astype(str) + " (" + grouped["percentage"].round(1).astype(str) + "%)"

    # Create bar chart
    fig = go.Figure()

    for resident in grouped["resident"].unique():
        df_res = grouped[grouped["resident"] == resident]
        fig.add_trace(go.Bar(
            x=df_res["country"],
            y=df_res["count"],
            name=resident,
            text=df_res["text"],
            textposition="outside"
        ))

    fig.update_layout(
        barmode='group',
        title=f"{ground} Inadmissibility Grounds by Residency Type (Top 5 Countries)",
        xaxis_title="Country",
        yaxis_title="Count",
        width=1100,
        height=700
    )

    fig.show()


    time_grouped = df_top.groupby(["year", "country"])["count"].sum().reset_index()

    fig_ts = px.line(
        time_grouped,
        x="year",
        y="count",
        markers=True,
        title=f"Time Series of {ground} Cases by Country (Top 5 Countries)",
        facet_col="country",
        facet_col_wrap=3
    )

    fig_ts.update_layout(
        yaxis_title="Count",
        xaxis_title="Year",
        height=800,
        width=1300
    )

    fig_ts.show()


Generating visualizations for A34(1)(f)...


Generating visualizations for A34(1)(a)...


Generating visualizations for A34(1)(b)...


Generating visualizations for A34(1)(c)...


Generating visualizations for A34(1)(d)...


# EDA Part 3

### A34(1) Refusal Trends (Top 5 Countries, Exclude Ukraine and Syria)

In [72]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Exclude Ukraine and Syria
filtered_df = df[~df['country'].isin(['Ukraine', 'Syria'])]

# Aggregate total refusals by country and year
grouped = filtered_df.groupby(['country', 'year'])['count'].sum().reset_index()

# Calculate total refusals for each country and select the top 5
top5_countries = (
    grouped.groupby('country')['count']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist()
)

# Keep data only for the top 5 countries
top5_grouped = grouped[grouped['country'].isin(top5_countries)]

# Create subplot layout: 3 rows and 2 columns
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=top5_countries,
    shared_xaxes=False,
    shared_yaxes=False
)

# Plot a separate chart for each country
row, col = 1, 1
for country in top5_countries:
    country_data = top5_grouped[top5_grouped['country'] == country]
    fig.add_trace(
        go.Scatter(x=country_data['year'], y=country_data['count'], mode='lines+markers', name=country),
        row=row, col=col
    )
    fig.update_yaxes(range=[0, 30], row=row, col=col)
    # Update subplot position
    col += 1
    if col > 2:
        col = 1
        row += 1

# Configure overall chart layout
fig.update_layout(
    height=800,
    width=900,
    title_text="A34(1) Refusal Trends (Top 5 Countries Exclude Ukraine and Syria)",
    showlegend=False
)

# Set axis titles
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Refusal Count")

fig.show()


**Iran** has shown a steadily increasing trend in refusal counts, especially from 2021 to 2024, with three consecutive years of growth. This indicates that more cases from Iran are being classified under the A34(1) ground as security risks.

**Bangladesh** experienced an unusually high number of refusals in 2019—the highest among the five countries. This was followed by a sharp decline over the next few years, then a gradual rebound, forming a clear “V-shaped” recovery trend.

**China** exhibited a fluctuating upward trend. After reaching a low point in 2022, refusal numbers surged significantly in 2023 and 2024, reaching one of the highest values in the chart. This may reflect systemic factors or policy shifts.

**Eritrea** maintained relatively low refusal counts overall but showed an upward trend, with a notable increase in 2024. Although the absolute numbers are small, the steady rise suggests a growing perception of risk.

**Ethiopia** displayed a relatively stable pattern with fewer refusals than other countries. However, the past two years have seen a gradual increase, and the 2024 figure is the highest in the six-year period, signaling a potential emerging upward trend.


### Dumbbell Chart: Top 5 Countries with Largest A34(1) Refusal Increases (2019 vs 2024)

In [73]:
# Filter data for years 2019 and 2024
dumbbell_df = df[df['year'].isin([2019, 2024])]

# Aggregate total refusals by country and year
pivot_df = dumbbell_df.groupby(['country', 'year'])['count'].sum().unstack()

# Drop rows with missing values and rename columns
pivot_df = pivot_df.dropna().reset_index()
pivot_df.columns = ['country', 'T0', 'T4']  # T0 = 2019, T4 = 2024

# Calculate change and select top 5 countries with the largest increases
pivot_df['change'] = pivot_df['T4'] - pivot_df['T0']
top_changes = pivot_df.sort_values(by='change', ascending=False).head(5)

# Plot the dumbbell chart
fig = go.Figure()

for i, row in top_changes.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['T0'], row['T4']],
        y=[row['country'], row['country']],
        mode="lines+markers+text",
        line=dict(color="gray", width=2),
        marker=dict(size=10, color=["blue", "orange"]),
        name=row["country"],
        showlegend=False,
        text=[f"2019: {row['T0']}", f"2024: {row['T4']}"],
        textposition="top center"
    ))

# Configure chart layout
fig.update_layout(
    title="Top 5 Countries with Largest A34(1) Refusal Increases (2019 vs 2024)",
    xaxis_title="Refusal Count",
    yaxis_title="Country",
    height=600,
    xaxis=dict(range=[0, top_changes[['T0', 'T4']].values.max() + 10]),
    yaxis=dict(
        categoryorder="array",
        categoryarray=top_changes["country"].tolist()
    )
)

fig.show()


This chart uses a Dumbbell Plot to illustrate the five countries with the largest increases in A34(1) refusal counts between 2019 and 2024. It clearly highlights how each country's risk profile, as perceived by Canada's immigration system, has evolved during this period.

Ukraine experienced the most dramatic surge in refusals, rising from 28 in 2019 to 131 in 2024 — the highest total and the largest increase among all countries. This sharp rise likely reflects the influence of war, geopolitical instability, or intelligence sharing on security assessments.

China and Iran also show steady increases, with refusals growing from 5 to 20 and from 8 to 18, respectively. This suggests a continued elevation in the sensitivity surrounding applicants from these countries under security grounds.

Although Sri Lanka and Afghanistan had lower overall numbers, both countries saw their refusal counts climb from single to double digits, indicating that risk detection for cases from these regions is also intensifying.


### Top 5 Countries A34(1) Refusals (2019 vs 2024 Slope Graph)

In [74]:
# Keep only data from 2019 and 2024
df_filtered = df[df['year'].isin([2019, 2024])]

# Summarize refusal counts per country
pivot_df = df_filtered.groupby(['country', 'year'])['count'].sum().unstack()

# Drop countries with missing data
pivot_df = pivot_df.dropna().reset_index()
pivot_df.columns = ['country', '2019', '2024']

# Select top 5 countries with the highest refusal counts in 2024
top5 = pivot_df.sort_values(by='2024', ascending=False).head(5)

# Create the slope graph
fig = go.Figure()

for _, row in top5.iterrows():
    fig.add_trace(go.Scatter(
        x=["2019", "2024"],
        y=[row["2019"], row["2024"]],
        mode="lines+markers+text",
        name=row["country"],
        text=[f"{row['country']}: {int(row['2019'])}", f"{row['country']}: {int(row['2024'])}"],
        textposition="middle right",
        line=dict(width=3)
    ))

# Layout settings: white background + legend shown
fig.update_layout(
    title="Top 5 Countries A34(1) Refusals (2019 vs 2024 Slope Graph)",
    xaxis=dict(title="", tickvals=["2019", "2024"]),
    yaxis_title="Refusal Count",
    height=800,
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=True
)

# Add vertical reference lines at 2019 and 2024
fig.add_vline(x=0, line_width=1, line_color="lightgray")  # 2019
fig.add_vline(x=1, line_width=1, line_color="lightgray")  # 2024

# Display the chart
fig.show()

This chart illustrates the change in A34(1) refusal counts for five countries between 2019 and 2024 using a slope graph format, clearly visualizing the before-and-after comparison.

Key observations from the chart:

* **Ukraine** shows a dramatic increase in refusals, jumping from 28 cases in 2019 to 131 in 2024. It stands out as the only country with a sharply rising trend.
* **China** increased from 5 to 20 cases, and **Iran** from 8 to 18, both reflecting a steady upward trajectory, suggesting intensified security screening for applicants from these countries.
* **Sri Lanka**, despite starting from the lowest point (3), rose to 16, showing relatively rapid growth.
* **Bangladesh** is the only country to show a decline, with refusal counts dropping from 24 in 2019 to 16 in 2024. This downward slope may indicate a shift in screening criteria or evolving risk assessment.
